In [210]:
%matplotlib inline
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk


In [ ]:
file_path = '/data/opinion_spam/real/reviewsNew.txt-utf8'
datas = []
MIN_FD = 100
raw_file = open(file_path)
debug = False
debug_size = 100
_debug_count = 0
for line in raw_file:    
    try:
        line = line.decode('gbk')
    except:
        continue
    #print 'gbk', line
    line = line.encode('utf-8')
    #print 'utf8', line
    item = line.split('\t')
    if len(item) == 8:
        hfd = float(item[3])
        fd = float(item[4])
        ratings = float(item[5])
        if fd > MIN_FD:
            hfdfd = round(hfd/fd, 2)
            #whole_filted_body += item[7]
            item.append(hfdfd)
            datas.append(item)
            if debug == True:
                _debug_count += 1
                if _debug_count >= debug_size:
                    break
raw_file.close()
print 'datas.size: ', len(datas)
print datas[0]

In [ ]:
#set column names
column_names = ['rid', 'pid', 'date', 'hfd', 'fd', 'rating', 'title', 'body', 'hfdfd']
df = pd.DataFrame(datas)
df.columns = column_names
df.head()

In [ ]:
#transfer the type of column
df[['hfd', 'fd','rating']] = df[['hfd', 'fd', 'rating']].astype(float)

In [ ]:
df.describe()

In [ ]:
#hfdfd distribution
df.hfdfd.hist()
plt.title='The Distribute of hfd/fd'
plt.xlabel('Score of hfd/fd')
plt.ylabel('Count')

In [ ]:
#rating distribution
df.rating.hist()
plt.xlabel('Ratings')
plt.ylabel("Count")

In [ ]:
#count how many products in the dataset  ==》9809 products
df.groupby('pid').count().shape
print 'different product size:', df.groupby('pid').count().shape[0] 

In [ ]:
df.shape

In [ ]:
df['title_len'] = df['title'].apply(len)
df['body_len'] = df['body'].apply(len)

In [ ]:
df.head()

In [ ]:
df['title_len'].hist()
plt.xlabel('length of review title')
plt.ylabel('counts')

In [ ]:
df[df['body_len'] > 10][df['body_len'] < 5000]['body_len'].hist()
plt.xlim(0, 5000)
plt.xlabel("length of review body")
plt.ylabel("counts")

In [ ]:
df[['title_len', 'body_len']].describe()

# TFIDF features

In [ ]:
#TFIDF feature without lsa

import evaluation_self as es
max_features_count = 10000
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_df=0.5, max_features=max_features_count,
                                 min_df=2, stop_words='english', tokenizer = es.tokenize_and_stem)

tfidf_matrix = vectorizer.fit_transform(df.body)
print "the tfidf matrix size : ",tfidf_matrix.shape
print type(tfidf_matrix)


In [ ]:
#TFIDF-CV-10-lsvr
from sklearn.svm import LinearSVR

lsvr = LinearSVR()

from sklearn.cross_validation import cross_val_score

scores2 = cross_val_score(lsvr, tfidf_matrix, df.hfdfd, cv=10, scoring="mean_squared_error")

mse2 = abs(scores2)
print 'TFIDF-cv10-LSVR, mse:', mse2
print 'TFIDF-cv10-LSVR, mse mean:', np.mean(mse2)
print 'TFIDF-cv10-LSVR, rmse:', np.sqrt(mse2)
print 'TFIDF-cv10-LSVR, rmse mean:', np.mean(np.sqrt(mse2))

In [ ]:
#TFIDF-CV-10-svr
from sklearn.svm import SVR

svr = SVR()

from sklearn.cross_validation import cross_val_score

scores2 = cross_val_score(svr, tfidf_matrix, df.hfdfd, cv=10, scoring="mean_squared_error")

mse2 = abs(scores2)
print 'TFIDF-cv10-SVR, mse:', mse2
print 'TFIDF-cv10-SVR, mse mean:', np.mean(mse2)
print 'TFIDF-cv10-SVR, rmse:', np.sqrt(mse2)
print 'TFIDF-cv10-SVR, rmse mean:', np.mean(np.sqrt(mse2))

In [ ]:
#TFIDF-CV-10-lr
from sklearn.linear_model import LinearRegression

lr = LinearRegression()

from sklearn.cross_validation import cross_val_score

scores2 = cross_val_score(lr, tfidf_matrix, df.hfdfd, cv=10, scoring="mean_squared_error")

mse2 = abs(scores2)
print 'TFIDF-cv10-LR, mse:', mse2
print 'TFIDF-cv10-LR, mse mean:', np.mean(mse2)
print 'TFIDF-cv10-LR, rmse:', np.sqrt(mse2)
print 'TFIDF-cv10-LR, rmse mean:', np.mean(np.sqrt(mse2))

# TFIDF - LSA features

In [ ]:
#TFIDF feature without lsa
from time import time
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

import evaluation_self as es
max_features_count = 10000
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_df=0.5, max_features=max_features_count,
                                 min_df=2, stop_words='english', tokenizer = es.tokenize_and_stem)
lsa_tfidf_matrix = vectorizer.fit_transform(df.body)
print "the tfidf matrix size : ",tfidf_matrix.shape
print type(tfidf_matrix)

lsaornot = True
lsanum = 1000

if lsaornot:
    print "Performing dimensionality reduction using LSA"
    t0 = time()
    # Vectorizer results are normalized, which makes KMeans behave as
    # spherical k-means for better results. Since LSA/SVD results are
    # not normalized, we have to redo the normalization.
    svd = TruncatedSVD(lsanum)
    normalizer = Normalizer(copy=False)
    lsa = make_pipeline(svd, normalizer)

    lsa_tfidf_matrix = lsa.fit_transform(tfidf_matrix)

    print "done in ", (time() - t0)
    explained_variance = svd.explained_variance_ratio_.sum()
    print "Explained variance of the SVD step:", int(explained_variance * 100)


In [ ]:
#TFIDF-LSA-cv10-LR
from sklearn.linear_model import LinearRegression

lr = LinearRegression()

from sklearn.cross_validation import cross_val_score

scores2 = cross_val_score(lr, lsa_tfidf_matrix, df.hfdfd, cv=10, scoring="mean_squared_error")

mse2 = abs(scores2)
print 'TFIDF-LSA-cv10-LR, mse:', mse2
print 'TFIDF-LSA-cv10-LR, mse mean:', np.mean(mse2)
print 'TFIDF-LSA-cv10-LR, rmse2:', np.sqrt(mse2)
print 'TFIDF-LSA-cv10-LR, rmse2 mean:', np.mean(np.sqrt(mse2))

In [ ]:
#TFIDF-LSA-CV-10-lsvr
from sklearn.svm import LinearSVR

lsvr = LinearSVR()

from sklearn.cross_validation import cross_val_score

scores2 = cross_val_score(lsvr, lsa_tfidf_matrix, df.hfdfd, cv=10, scoring="mean_squared_error")

mse2 = abs(scores2)
print 'TFIDF-cv10-LSVR, mse:', mse2
print 'TFIDF-cv10-LSVR, mse mean:', np.mean(mse2)
print 'TFIDF-cv10-LSVR, rmse:', np.sqrt(mse2)
print 'TFIDF-cv10-LSVR, rmse mean:', np.mean(np.sqrt(mse2))

In [ ]:
#TFIDF-LSA-CV-10-svr
t0 = time()
from sklearn.svm import SVR

svr = SVR()

from sklearn.cross_validation import cross_val_score

scores2 = cross_val_score(svr, lsa_tfidf_matrix, df.hfdfd, cv=10, scoring="mean_squared_error")

mse2 = abs(scores2)
print 'TFIDF-cv10-SVR, mse:', mse2
print 'TFIDF-cv10-SVR, mse mean:', np.mean(mse2)
print 'TFIDF-cv10-SVR, rmse:', np.sqrt(mse2)
print 'TFIDF-cv10-SVR, rmse mean:', np.mean(np.sqrt(mse2))
print "tfidf-svr cost:", (time() - t0)

# STD features

In [ ]:
#STD features

import nltk
import re

def seg_sentence(text):
    sentences = nltk.sent_tokenize(text)
    return [s.strip() for s in sentences if(len(s)) > 0]

def marks_num(text, pattern):
    return len(re.findall(pattern, text))

std_exc = 0
def std_feature(text): # std in Yang@acl-2015
#    print '==============================='
    ary = [0, 0, 0, 0, 0]
    try:
        tokens = es.tokenize_and_stem(text)
        token_num = len(tokens)
    #    print 'token size:', token_num
        sentences = seg_sentence(text)
        sen_num = len(sentences)
    #    print 'sentence size:', sen_num
        avg_sentence_len = np.mean([len(x) for x in sentences])
    #    print 'avg sentence length:', avg_sentence_len
        exc_num = marks_num(text, r'[!！]')
    #    print 'exclamation marks count:', exc_num
        que_num = marks_num(text, r'[？?]')
    #    print 'question percentage:', float(que_num) / sen_num
        ary = [token_num, sen_num, avg_sentence_len, exc_num, que_num]
#         print ary
    except:
#         print text
        std_exc += 1
    finally:
        return ary
    
for loc in range(1, 10):
    print std(df.body.head(10)[loc])

In [ ]:
#generate std features


X_std = np.array(df.body)
print type(X_std)
X_std = np.array(map(std_feature, X_std))
print "std exception: ", std_exc

print type(X_std)
print X_std.shape

from sklearn.preprocessing import Normalizer
std_normalizer = Normalizer().fit(X_std)
X_std = std_normalizer.transform(X_std)


In [ ]:
#std-cv10-lr
from sklearn.linear_model import LinearRegression
std_lr = LinearRegression()

std_scores = cross_val_score(std_lr, X_std, df.hfdfd, cv=10, scoring='mean_squared_error')

std_mse = abs(std_scores)
print 'STD-cv10-LR, mse:', std_mse
print 'STD-cv10-LR, mse mean:', np.mean(std_mse)
print 'STD-cv10-LR, rmse:', np.sqrt(std_mse)
print 'STD-cv10-LR, rmse mean:', np.mean(np.sqrt(std_mse))

In [ ]:
#std-cv10-lsvr
from sklearn.svm import LinearSVR
std_lsvr = LinearSVR()

std_scores = cross_val_score(std_lsvr, X_std, df.hfdfd, cv=10, scoring='mean_squared_error')

std_mse = abs(std_scores)
print 'STD-cv10-LSVR, mse:', std_mse
print 'STD-cv10-LSVR, mse mean:', np.mean(std_mse)
print 'STD-cv10-LSVR, rmse:', np.sqrt(std_mse)
print 'STD-cv10-LSVR, rmse mean:', np.mean(np.sqrt(std_mse))

In [ ]:
#std-cv10-svr
t0 = time()
from sklearn.svm import SVR
std_svr = SVR()

std_scores = cross_val_score(std_svr, X_std, df.hfdfd, cv=10, scoring='mean_squared_error')

std_mse = abs(std_scores)
print 'STD-cv10-SVR, mse:', std_mse
print 'STD-cv10-SVR, mse mean:', np.mean(std_mse)
print 'STD-cv10-SVR, rmse:', np.sqrt(std_mse)
print 'STD-cv10-SVR, rmse mean:', np.mean(np.sqrt(std_mse))
print "STD-svr cost:", (time() - t0)

In [ ]:
#def cut sentence
stop_words = 'a,able,about,across,after,all,almost,also,am,among,an,and,any,are,as,at,be,because,been,but,by,can,cannot,could,dear,did,do,does,either,else,ever,every,for,from,get,got,had,has,have,he,her,hers,him,his,how,however,i,if,in,into,is,it,its,just,least,let,like,likely,may,me,might,most,must,my,neither,no,nor,not,of,off,often,on,only,or,other,our,own,rather,said,say,says,she,should,since,so,some,than,that,the,their,them,then,there,these,they,this,tis,to,too,twas,us,wants,was,we,were,what,when,where,which,while,who,whom,why,will,with,would,yet,you,your'
stop_words_list = stop_words.split(',')

def cut_into_tokens(text):

    _list = [word.lower() for sentence in nltk.sent_tokenize(text) for word in sentence.split()]
    _list = [re.sub(r'[.?!\'",)(;&%:]', '', word) for word in _list  ]
    _list = [word for word in _list if word not in stop_words_list and len(word) > 0]
    return _list

# Syntactic features of Kim

In [ ]:
#Syntactic features of Kim

text = df.body.head(1)[0]
print text
#print nltk.pos_tag([word  for sentence in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sentence)])

def pos_count(text):
    poses = nltk.pos_tag(cut_into_tokens(text))
#     poses = nltk.pos_tag([word  for sentence in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sentence)])
    pos_dict = {}
    for (word, pos) in poses:
        if pos in pos_dict:
            pos_dict[pos] = pos_dict[pos] + 1
        else:
            pos_dict[pos] = 1
    return pos_dict

pos_dict = pos_count(text)#per sentence per second!!!!
# for pos in pos_dict:
#     print pos, " => ", pos_dict[pos]
# print 'aa'
syn_exc = 0

def fetch_count_from_pos_dict(pos_list, pos_dict):
    _pos_count = 0
    for pos in pos_list:
        if pos in pos_dict:
            _pos_count += pos_dict[pos]
    return _pos_count

def syntactic_feature(text): #syn feature function
    ary = [0, 0, 0, 0]
    try:
        pos_dict = pos_count(text)
        nn_count = fetch_count_from_pos_dict(['NN', 'NNS', 'NNP', 'NNPS'], pos_dict)#noun
        jj_count = fetch_count_from_pos_dict(['JJ', 'JJR', 'JJS'], pos_dict)#adj
        vb_count = fetch_count_from_pos_dict(['VB', 'VBD', 'VBG', 'VBN', 'VNP', 'VBZ'], pos_dict)#vb
        rb_count = fetch_count_from_pos_dict(['RB', 'RBR', 'RBS'], pos_dict)#adv
        ary = [nn_count, jj_count, vb_count, rb_count]
    except:
        print 'error'
        syn_exc += 1
    finally:
        return ary
# print 't_dict', pos_dict['_t_dict']
for _text in df.body.head(10):
    print syntactic_feature(_text)
print 'syn_exc is ', syn_exc

In [ ]:
X_syn = []
for _body in df.body:
    _v = syntactic_feature(_body)
    X_syn.append(_v)
print len(X_syn)

from sklearn.preprocessing import Normalizer

syn_normalizer = Normalizer()
syn_normalizer.fit(X_syn)
X_syn = syn_normalizer.transform(X_syn)

# print X_w2v
print syn_normalizer

In [ ]:
#syn-cv-10-LR
from sklearn.linear_model import LinearRegression

syn_lr = LinearRegression()

std_scores = cross_val_score(syn_lr, X_syn, df.hfdfd, cv=10, scoring='mean_squared_error')

std_mse = abs(std_scores)
print 'SYN-cv10-LR, mse:', std_mse
print 'SYN-cv10-LR, mse mean:', np.mean(std_mse)
print 'SYN-cv10-LR, rmse:', np.sqrt(std_mse)
print 'SYN-cv10-LR, rmse mean:', np.mean(np.sqrt(std_mse))

In [ ]:
#syn-cv-10-LSVR
from sklearn.svm import LinearSVR

syn_lsvr = LinearSVR()

std_scores = cross_val_score(syn_lsvr, X_syn, df.hfdfd, cv=10, scoring='mean_squared_error')

std_mse = abs(std_scores)
print 'SYN-cv10-LSVR, mse:', std_mse
print 'SYN-cv10-LSVR, mse mean:', np.mean(std_mse)
print 'SYN-cv10-LSVR, rmse:', np.sqrt(std_mse)
print 'SYN-cv10-LSVR, rmse mean:', np.mean(np.sqrt(std_mse))

In [ ]:
#syn-cv-10-SVR
t0 = time()
from sklearn.svm import SVR

syn_lsvr = SVR()

std_scores = cross_val_score(syn_lsvr, X_syn, df.hfdfd, cv=10, scoring='mean_squared_error')

std_mse = abs(std_scores)
print 'SYN-cv10-SVR, mse:', std_mse
print 'SYN-cv10-SVR, mse mean:', np.mean(std_mse)
print 'SYN-cv10-SVR, rmse:', np.sqrt(std_mse)
print 'SYN-cv10-SVR, rmse mean:', np.mean(np.sqrt(std_mse))
print "SYN-svr cost:", (time() - t0)

# Word2Vec features

In [ ]:
#w2v features

from gensim import corpora, models, similarities
import gensim

#load model
fname = '/data/opinion_spam/real/w2v-hfd-over-0.total'
#gensim.models.Word2Vec(sentences, min_count=5, size=100, window=5, workers=32)
model = gensim.models.Word2Vec.load(fname)
model.init_sims(replace=True)

In [ ]:
def model_with_sentence(sentence, model):
    tokens = cut_into_tokens(sentence)
    v = np.zeros(100)
    _error = 0
    for token in tokens:
        try:
            _v = model[token]
            v += _v
        except:
            _error += 1
#             print 'token:', token, 'not in the dict'
    return v
print 'def model_with_sentence'

In [ ]:
X_w2v = []
for _body in df.body:
    _v = model_with_sentence(_body, model)
    X_w2v.append(_v)
print len(X_w2v)

from sklearn.preprocessing import Normalizer

w2v_normalizer = Normalizer()
w2v_normalizer.fit(X_w2v)
X_w2v = w2v_normalizer.transform(X_w2v)

# print X_w2v
print w2v_normalizer

In [ ]:
#w2v-cv-10-LR
from sklearn.linear_model import LinearRegression

w2v_lr = LinearRegression()

std_scores = cross_val_score(w2v_lr, X_w2v, df.hfdfd, cv=10, scoring='mean_squared_error')

std_mse = abs(std_scores)
print 'W2V-cv10-LR, mse:', std_mse
print 'W2V-cv10-LR, mse mean:', np.mean(std_mse)
print 'W2V-cv10-LR, rmse:', np.sqrt(std_mse)
print 'W2V-cv10-LR, rmse mean:', np.mean(np.sqrt(std_mse))

In [ ]:
#w2v-cv-10-LSVR
from sklearn.svm import LinearSVR

w2v_lsvr = LinearSVR()

std_scores = cross_val_score(w2v_lsvr, X_w2v, df.hfdfd, cv=10, scoring='mean_squared_error')

std_mse = abs(std_scores)
print 'W2V-cv10-LSVR, mse:', std_mse
print 'W2V-cv10-LSVR, mse mean:', np.mean(std_mse)
print 'W2V-cv10-LSVR, rmse:', np.sqrt(std_mse)
print 'W2V-cv10-LSVR, rmse mean:', np.mean(np.sqrt(std_mse))

In [ ]:
#w2v-cv-10-SVR
t0 = time()
from sklearn.svm import SVR

w2v_svr = SVR()

std_scores = cross_val_score(w2v_svr, X_w2v, df.hfdfd, cv=10, scoring='mean_squared_error')

std_mse = abs(std_scores)
print 'W2V-cv10-SVR, mse:', std_mse
print 'W2V-cv10-SVR, mse mean:', np.mean(std_mse)
print 'W2V-cv10-SVR, rmse:', np.sqrt(std_mse)
print 'W2V-cv10-SVR, rmse mean:', np.mean(np.sqrt(std_mse))
print "W2V-svr cost:", (time() - t0)

# Utils for merge features

In [ ]:
def matrix_to_array(_matrix):
    _dense = _matrix.todense()
    _list = _dense.tolist()
    return _list

def append_matrix_to_matrix(array1, array2):
    length = len(array1)#row num
    new_matrix = []
    for index in range(length):
        _row = np.append(array1[index], array2[index])
        new_matrix.append(_row)
    return new_matrix

In [ ]:
#validate the matrix merge
ar1 = [[1,2,3],[2,3,4]]
ar2 = [[5,6,7], [7,8,9]]
ar3 = append_matrix_to_matrix(ar1, ar2)
print ar3

# STD + Word2Vec features

In [ ]:
#combine std + w2v
# X_std_w2v = []
# for _body in df.body:
#     _w2v = model_with_sentence(_body, model)
#     _std = std(_body)
#     _v = np.append(_w2v, _std)
#     X_std_w2v.append(_v)
# print 'feature size : ', len(X_std_w2v[0])

# std_w2v_normalizer = Normalizer()
# std_w2v_normalizer.fit(X_w2v)
# X_std_w2v = std_w2v_normalizer.transform(X_w2v)

# print X_w2v
# print std_w2v_normalizer

X_std_w2v = append_matrix_to_matrix(X_std, X_w2v)

In [ ]:
#std+w2v-cv-10-LR
from sklearn.linear_model import LinearRegression

std_w2v_lr = LinearRegression()

std_scores = cross_val_score(std_w2v_lr, X_std_w2v, df.hfdfd, cv=10, scoring='mean_squared_error')

std_mse = abs(std_scores)
print 'STD+W2V-cv10-LR, mse:', std_mse
print 'STD+W2V-cv10-LR, mse mean:', np.mean(std_mse)
print 'STD+W2V-cv10-LR, rmse:', np.sqrt(std_mse)
print 'STD+W2V-cv10-LR, rmse mean:', np.mean(np.sqrt(std_mse))

In [ ]:
#std+w2v-cv-10-LSVR
from sklearn.svm import LinearSVR

std_w2v_lsvr = LinearSVR()

std_scores = cross_val_score(std_w2v_lsvr, X_std_w2v, df.hfdfd, cv=10, scoring='mean_squared_error')

std_mse = abs(std_scores)
print 'STD+W2V-cv10-LSVR, mse:', std_mse
print 'STD+W2V-cv10-LSVR, mse mean:', np.mean(std_mse)
print 'STD+W2V-cv10-LSVR, rmse:', np.sqrt(std_mse)
print 'STD+W2V-cv10-LSVR, rmse mean:', np.mean(np.sqrt(std_mse))

In [ ]:
#std+w2v-cv-10-SVR
t0 = time()
from sklearn.svm import SVR

std_w2v_svr = SVR()

std_scores = cross_val_score(std_w2v_svr, X_std_w2v, df.hfdfd, cv=10, scoring='mean_squared_error')

std_mse = abs(std_scores)
print 'STD+W2V-cv10-SVR, mse:', std_mse
print 'STD+W2V-cv10-SVR, mse mean:', np.mean(std_mse)
print 'STD+W2V-cv10-SVR, rmse:', np.sqrt(std_mse)
print 'STD+W2V-cv10-SVR, rmse mean:', np.mean(np.sqrt(std_mse))
print "STD+W2V-svr cost:", (time() - t0)

# STD + SYN features

In [ ]:
# X_std_syn = []
# for _body in df.body:
#     _std = std(_body)
#     _syn = syntactic_feature(_body)
#     _v = np.append(_std, _syn)
#     X_std_syn.append(_v)
# print 'feature size : ', len(X_std_syn[0])

# std_syn_normalizer = Normalizer()
# std_syn_normalizer.fit(X_std_syn)
# X_std_syn = std_syn_normalizer.transform(X_std_syn)

# # print X_w2v
# print std_syn_normalizer

X_std_syn = append_matrix_to_matrix(X_std, X_syn)

In [ ]:
#std+syn-cv-10-LR
from sklearn.linear_model import LinearRegression

std_syn_lr = LinearRegression()

std_scores = cross_val_score(std_syn_lr, X_std_syn, df.hfdfd, cv=10, scoring='mean_squared_error')

std_mse = abs(std_scores)
print 'STD+SYN-cv10-LR, mse:', std_mse
print 'STD+SYN-cv10-LR, mse mean:', np.mean(std_mse)
print 'STD+SYN-cv10-LR, rmse:', np.sqrt(std_mse)
print 'STD+SYN-cv10-LR, rmse mean:', np.mean(np.sqrt(std_mse))

In [ ]:
#std+syn-cv-10-LSVR
from sklearn.svm import LinearSVR

std_syn_lsvr = LinearSVR()

std_scores = cross_val_score(std_syn_lsvr, X_std_syn, df.hfdfd, cv=10, scoring='mean_squared_error')

std_mse = abs(std_scores)
print 'STD+SYN-cv10-LSVR, mse:', std_mse
print 'STD+SYN-cv10-LSVR, mse mean:', np.mean(std_mse)
print 'STD+SYN-cv10-LSVR, rmse:', np.sqrt(std_mse)
print 'STD+SYN-cv10-LSVR, rmse mean:', np.mean(np.sqrt(std_mse))

In [ ]:
#std+syn-cv-10-SVR
t0 = time()
from sklearn.svm import SVR

std_syn_svr = SVR()

std_scores = cross_val_score(std_syn_svr, X_std_syn, df.hfdfd, cv=10, scoring='mean_squared_error')

std_mse = abs(std_scores)
print 'STD+SYN-cv10-SVR, mse:', std_mse
print 'STD+SYN-cv10-SVR, mse mean:', np.mean(std_mse)
print 'STD+SYN-cv10-SVR, rmse:', np.sqrt(std_mse)
print 'STD+SYN-cv10-SVR, rmse mean:', np.mean(np.sqrt(std_mse))
print 'cost : ', time()-t0

# STD + SYN + W2V features

In [ ]:
# X_std_syn_w2v = []
# for _body in df.body:
#     _std = std(_body)
#     _syn = syntactic_feature(_body)
#     _w2v = model_with_sentence(_body, model)
#     _v = np.append(_std, _syn)
#     _v = np.append(_v, _w2v)
#     X_std_syn_w2v.append(_v)
# print 'feature size : ', len(X_std_syn_w2v[0])

# std_syn_w2v_normalizer = Normalizer()
# std_syn_w2v_normalizer.fit(X_std_syn)
# X_std_syn_w2v = std_syn_w2v_normalizer.transform(X_std_syn)

# # print X_w2v
# print std_syn_w2v_normalizer

X_std_syn_w2v = []
# X_std_syn_w2v = append_matrix_to_matrix(X_std_syn, X_w2v)
X_std_syn_w2v = append_matrix_to_matrix(X_std, X_w2v)
X_std_syn_w2v = append_matrix_to_matrix(X_std_syn_w2v, X_syn)
print 'row size: ',len(X_std_syn_w2v)
print 'column size: ',len(X_std_syn_w2v[0])

In [ ]:
#std+syn+w2v-cv-10-LR
from sklearn.linear_model import LinearRegression

std_syn_w2v_lr = LinearRegression()

std_scores = cross_val_score(std_syn_w2v_lr, X_std_syn_w2v, df.hfdfd, cv=10, scoring='mean_squared_error')

std_mse = abs(std_scores)
print 'STD+SYN+W2V-cv10-LR, mse:', std_mse
print 'STD+SYN+W2V-cv10-LR, mse mean:', np.mean(std_mse)
print 'STD+SYN+W2V-cv10-LR, rmse:', np.sqrt(std_mse)
print 'STD+SYN+W2V-cv10-LR, rmse mean:', np.mean(np.sqrt(std_mse))

In [ ]:
#std+syn+w2v-cv-10-LSVR
from sklearn.svm import LinearSVR

std_syn_w2v_lsvr = LinearSVR()

std_scores = cross_val_score(std_syn_w2v_lsvr, X_std_syn_w2v, df.hfdfd, cv=10, scoring='mean_squared_error')

std_mse = abs(std_scores)
print 'STD+SYN+W2V-cv10-LSVR, mse:', std_mse
print 'STD+SYN+W2V-cv10-LSVR, mse mean:', np.mean(std_mse)
print 'STD+SYN+W2V-cv10-LSVR, rmse:', np.sqrt(std_mse)
print 'STD+SYN+W2V-cv10-LSVR, rmse mean:', np.mean(np.sqrt(std_mse))

In [ ]:
#std+syn+w2v-cv-10-LSVR
from sklearn.svm import SVR

std_syn_w2v_svr = SVR()

std_scores = cross_val_score(std_syn_w2v_svr, X_std_syn_w2v, df.hfdfd, cv=10, scoring='mean_squared_error')

std_mse = abs(std_scores)
print 'STD+SYN+W2V-cv10-SVR, mse:', std_mse
print 'STD+SYN+W2V-cv10-SVR, mse mean:', np.mean(std_mse)
print 'STD+SYN+W2V-cv10-SVR, rmse:', np.sqrt(std_mse)
print 'STD+SYN+W2V-cv10-SVR, rmse mean:', np.mean(np.sqrt(std_mse))

# TFIDF + STD features

In [ ]:
#combine tfidf+std
X_tfidf_std = []
print 'tfidf_matrix type : ', type(tfidf_matrix)
X_tfidf_std = append_matrix_to_matrix(matrix_to_array(tfidf_matrix), X_std)
print 'row size: ',len(X_tfidf_std)
print 'column size: ',len(X_tfidf_std[0])

In [ ]:
#tfidf+std-cv-10-LR
from sklearn.linear_model import LinearRegression

tfidf_std_lr = LinearRegression()

std_scores = cross_val_score(tfidf_std_lr, X_tfidf_std, df.hfdfd, cv=10, scoring='mean_squared_error')

std_mse = abs(std_scores)
print 'TFIDF+STD-cv10-LR, mse:', std_mse
print 'TFIDF+STD-cv10-LR, mse mean:', np.mean(std_mse)
print 'TFIDF+STD-cv10-LR, rmse:', np.sqrt(std_mse)
print 'TFIDF+STD-cv10-LR, rmse mean:', np.mean(np.sqrt(std_mse))

In [ ]:
#tfidf+std-cv-10-LSVR
from sklearn.svm import LinearSVR

tfidf_std_lsvr = LinearSVR()

std_scores = cross_val_score(tfidf_std_lsvr, X_tfidf_std, df.hfdfd, cv=10, scoring='mean_squared_error')

std_mse = abs(std_scores)
print 'TFIDF+STD-cv10-LSVR, mse:', std_mse
print 'TFIDF+STD-cv10-LSVR, mse mean:', np.mean(std_mse)
print 'TFIDF+STD-cv10-LSVR, rmse:', np.sqrt(std_mse)
print 'TFIDF+STD-cv10-LSVR, rmse mean:', np.mean(np.sqrt(std_mse))

In [ ]:
#tfidf+std-cv-10-LSVR
from sklearn.svm import SVR

tfidf_std_svr = SVR()

std_scores = cross_val_score(tfidf_std_svr, X_tfidf_std, df.hfdfd, cv=10, scoring='mean_squared_error')

std_mse = abs(std_scores)
print 'TFIDF+STD-cv10-SVR, mse:', std_mse
print 'TFIDF+STD-cv10-SVR, mse mean:', np.mean(std_mse)
print 'TFIDF+STD-cv10-SVR, rmse:', np.sqrt(std_mse)
print 'TFIDF+STD-cv10-SVR, rmse mean:', np.mean(np.sqrt(std_mse))

# TFIDF + STD + SYN features

In [ ]:
X_tfidf_std_syn = []
X_tfidf_std_syn = append_matrix_to_matrix(X_tfidf_std, X_syn)
print 'row size: ',len(X_tfidf_std_syn)
print 'column size: ',len(X_tfidf_std_syn[0])

In [ ]:
#tfidf+std+syn-cv-10-LR
from sklearn.linear_model import LinearRegression

tfidf_std_syn_lr = LinearRegression()

std_scores = cross_val_score(tfidf_std_syn_lr, X_tfidf_std_syn, df.hfdfd, cv=10, scoring='mean_squared_error')

std_mse = abs(std_scores)
print 'TFIDF+STD+SYN-cv10-LR, mse:', std_mse
print 'TFIDF+STD+SYN-cv10-LR, mse mean:', np.mean(std_mse)
print 'TFIDF+STD+SYN-cv10-LR, rmse:', np.sqrt(std_mse)
print 'TFIDF+STD+SYN-cv10-LR, rmse mean:', np.mean(np.sqrt(std_mse))

In [ ]:
#tfidf+std-cv-10-LSVR
from sklearn.svm import LinearSVR

tfidf_std_syn_lsvr = LinearRegression()

std_scores = cross_val_score(tfidf_std_syn_lsvr, X_tfidf_std_syn, df.hfdfd, cv=10, scoring='mean_squared_error')

std_mse = abs(std_scores)
print 'TFIDF+STD+SYN-cv10-LSVR, mse:', std_mse
print 'TFIDF+STD+SYN-cv10-LSVR, mse mean:', np.mean(std_mse)
print 'TFIDF+STD+SYN-cv10-LSVR, rmse:', np.sqrt(std_mse)
print 'TFIDF+STD+SYN-cv10-LSVR, rmse mean:', np.mean(np.sqrt(std_mse))

In [ ]:
#tfidf+std-cv-10-LSVR
from sklearn.svm import SVR

tfidf_std_syn_svr = SVR()

std_scores = cross_val_score(tfidf_std_syn_svr, X_tfidf_std_syn, df.hfdfd, cv=10, scoring='mean_squared_error')

std_mse = abs(std_scores)
print 'TFIDF+STD+SYN-cv10-SVR, mse:', std_mse
print 'TFIDF+STD+SYN-cv10-SVR, mse mean:', np.mean(std_mse)
print 'TFIDF+STD+SYN-cv10-SVR, rmse:', np.sqrt(std_mse)
print 'TFIDF+STD+SYN-cv10-SVR, rmse mean:', np.mean(np.sqrt(std_mse))

# TFIDF + STD + SYN + W2V features

In [ ]:
X_tfidf_std_syn_w2v = []
X_tfidf_std_syn_w2v = append_matrix_to_matrix(X_tfidf_std_syn, X_w2v)
print 'row size: ',len(X_tfidf_std_syn_w2v)
print 'column size: ',len(X_tfidf_std_syn_w2v[0])

In [ ]:
#tfidf+std+syn-cv-10-LR
from sklearn.linear_model import LinearRegression

tfidf_std_syn_w2v_lr = LinearRegression()

std_scores = cross_val_score(tfidf_std_syn_w2v_lr, X_tfidf_std_syn_w2v, df.hfdfd, cv=10, scoring='mean_squared_error')

std_mse = abs(std_scores)
print 'TFIDF+STD+SYN+W2V-cv10-LR, mse:', std_mse
print 'TFIDF+STD+SYN+W2V-cv10-LR, mse mean:', np.mean(std_mse)
print 'TFIDF+STD+SYN+W2V-cv10-LR, rmse:', np.sqrt(std_mse)
print 'TFIDF+STD+SYN+W2V-cv10-LR, rmse mean:', np.mean(np.sqrt(std_mse))

In [ ]:
#tfidf+std+syn-cv-10-LR
from sklearn.svm import LinearSVR

tfidf_std_syn_w2v_lsvr = LinearSVR()

std_scores = cross_val_score(tfidf_std_syn_w2v_lsvr, X_tfidf_std_syn_w2v, df.hfdfd, cv=10, scoring='mean_squared_error')

std_mse = abs(std_scores)
print 'TFIDF+STD+SYN+W2V-cv10-LSVR, mse:', std_mse
print 'TFIDF+STD+SYN+W2V-cv10-LSVR, mse mean:', np.mean(std_mse)
print 'TFIDF+STD+SYN+W2V-cv10-LSVR, rmse:', np.sqrt(std_mse)
print 'TFIDF+STD+SYN+W2V-cv10-LSVR, rmse mean:', np.mean(np.sqrt(std_mse))

In [ ]:
#tfidf+std+syn-cv-10-SVR
from sklearn.svm import SVR

tfidf_std_syn_w2v_svr = SVR()

std_scores = cross_val_score(tfidf_std_syn_w2v_svr, X_tfidf_std_syn_w2v, df.hfdfd, cv=10, scoring='mean_squared_error')

std_mse = abs(std_scores)
print 'TFIDF+STD+SYN+W2V-cv10-SVR, mse:', std_mse
print 'TFIDF+STD+SYN+W2V-cv10-SVR, mse mean:', np.mean(std_mse)
print 'TFIDF+STD+SYN+W2V-cv10-SVR, rmse:', np.sqrt(std_mse)
print 'TFIDF+STD+SYN+W2V-cv10-SVR, rmse mean:', np.mean(np.sqrt(std_mse))

# TFIDF+LSA + STD features

In [ ]:
#combine tfidf+std
X_lsa_tfidf_std = []
print 'lsa_tfidf_matrix type: ', type(lsa_tfidf_matrix)
X_lsa_tfidf_std = append_matrix_to_matrix(lsa_tfidf_matrix, X_std)
print 'row size: ',len(X_lsa_tfidf_std)
print 'column size: ',len(X_lsa_tfidf_std[0])

In [ ]:
#lsa+tfidf+std-cv-10-LR
from sklearn.linear_model import LinearRegression

lsa_tfidf_std_lr = LinearRegression()

std_scores = cross_val_score(lsa_tfidf_std_lr, X_lsa_tfidf_std, df.hfdfd, cv=10, scoring='mean_squared_error')

std_mse = abs(std_scores)
print 'LSA+TFIDF+STD-cv10-LR, mse:', std_mse
print 'LSA+TFIDF+STD-cv10-LR, mse mean:', np.mean(std_mse)
print 'LSA+TFIDF+STD-cv10-LR, rmse:', np.sqrt(std_mse)
print 'LSA+TFIDF+STD-cv10-LR, rmse mean:', np.mean(np.sqrt(std_mse))

In [ ]:
#lsa+tfidf+std-cv-10-LSVR
from sklearn.svm import LinearSVR

lsa_tfidf_std_lsvr = LinearSVR()

std_scores = cross_val_score(lsa_tfidf_std_lsvr, X_lsa_tfidf_std, df.hfdfd, cv=10, scoring='mean_squared_error')

std_mse = abs(std_scores)
print 'LSA+TFIDF+STD-cv10-LSVR, mse:', std_mse
print 'LSA+TFIDF+STD-cv10-LSVR, mse mean:', np.mean(std_mse)
print 'LSA+TFIDF+STD-cv10-LSVR, rmse:', np.sqrt(std_mse)
print 'LSA+TFIDF+STD-cv10-LSVR, rmse mean:', np.mean(np.sqrt(std_mse))

In [ ]:
#lsa+tfidf+std-cv-10-SVR
from sklearn.svm import SVR

lsa_tfidf_std_lsvr = SVR()

std_scores = cross_val_score(lsa_tfidf_std_lsvr, X_lsa_tfidf_std, df.hfdfd, cv=10, scoring='mean_squared_error')

std_mse = abs(std_scores)
print 'LSA+TFIDF+STD-cv10-SVR, mse:', std_mse
print 'LSA+TFIDF+STD-cv10-SVR, mse mean:', np.mean(std_mse)
print 'LSA+TFIDF+STD-cv10-SVR, rmse:', np.sqrt(std_mse)
print 'LSA+TFIDF+STD-cv10-SVR, rmse mean:', np.mean(np.sqrt(std_mse))

# LSA+TFIDF + STD + SYN features

In [ ]:
X_lsa_tfidf_std_syn = []
X_lsa_tfidf_std_syn = append_matrix_to_matrix(X_lsa_tfidf_std, X_syn)
print 'row size: ',len(X_lsa_tfidf_std_syn)
print 'column size: ',len(X_lsa_tfidf_std_syn[0])

In [ ]:
#lsa+tfidf+std-cv-10-LR
from sklearn.linear_model import LinearRegression

lsa_tfidf_std_syn_lr = LinearRegression()

std_scores = cross_val_score(lsa_tfidf_std_syn_lr, X_lsa_tfidf_std_syn, df.hfdfd, cv=10, scoring='mean_squared_error')

std_mse = abs(std_scores)
print 'LSA+TFIDF+STD+SYN-cv10-LR, mse:', std_mse
print 'LSA+TFIDF+STD+SYN-cv10-LR, mse mean:', np.mean(std_mse)
print 'LSA+TFIDF+STD+SYN-cv10-LR, rmse:', np.sqrt(std_mse)
print 'LSA+TFIDF+STD+SYN-cv10-LR, rmse mean:', np.mean(np.sqrt(std_mse))

In [ ]:
#lsa+tfidf+std-cv-10-LSVR
from sklearn.svm import LinearSVR

lsa_tfidf_std_syn_lsvr = LinearSVR()

std_scores = cross_val_score(lsa_tfidf_std_syn_lsvr, X_lsa_tfidf_std_syn, df.hfdfd, cv=10, scoring='mean_squared_error')

std_mse = abs(std_scores)
print 'LSA+TFIDF+STD+SYN-cv10-LSVR, mse:', std_mse
print 'LSA+TFIDF+STD+SYN-cv10-LSVR, mse mean:', np.mean(std_mse)
print 'LSA+TFIDF+STD+SYN-cv10-LSVR, rmse:', np.sqrt(std_mse)
print 'LSA+TFIDF+STD+SYN-cv10-LSVR, rmse mean:', np.mean(np.sqrt(std_mse))

In [ ]:
#lsa+tfidf+std-cv-10-SVR
from sklearn.svm import SVR

lsa_tfidf_std_syn_svr = SVR()

std_scores = cross_val_score(lsa_tfidf_std_syn_svr, X_lsa_tfidf_std_syn, df.hfdfd, cv=10, scoring='mean_squared_error')

std_mse = abs(std_scores)
print 'LSA+TFIDF+STD+SYN-cv10-SVR, mse:', std_mse
print 'LSA+TFIDF+STD+SYN-cv10-SVR, mse mean:', np.mean(std_mse)
print 'LSA+TFIDF+STD+SYN-cv10-SVR, rmse:', np.sqrt(std_mse)
print 'LSA+TFIDF+STD+SYN-cv10-SVR, rmse mean:', np.mean(np.sqrt(std_mse))

# LSA+TFIDF + STD + SYN + W2V features

In [ ]:
X_lsa_tfidf_std_syn_w2v = []
X_lsa_tfidf_std_syn_w2v = append_matrix_to_matrix(X_lsa_tfidf_std_syn, X_w2v)
print 'row size: ',len(X_lsa_tfidf_std_syn_w2v)
print 'column size: ',len(X_lsa_tfidf_std_syn_w2v[0])

In [ ]:
#lsa+tfidf+std-cv-10-LR
from sklearn.linear_model import LinearRegression

lsa_tfidf_std_syn_w2v_lr = LinearRegression()

std_scores = cross_val_score(lsa_tfidf_std_syn_w2v_lr, X_lsa_tfidf_std_syn_w2v, df.hfdfd, cv=10, scoring='mean_squared_error')

std_mse = abs(std_scores)
print 'LSA+TFIDF+STD+SYN+W2V-cv10-LR, mse:', std_mse
print 'LSA+TFIDF+STD+SYN+W2V-cv10-LR, mse mean:', np.mean(std_mse)
print 'LSA+TFIDF+STD+SYN+W2V-cv10-LR, rmse:', np.sqrt(std_mse)
print 'LSA+TFIDF+STD+SYN+W2V-cv10-LR, rmse mean:', np.mean(np.sqrt(std_mse))

In [ ]:
#lsa+tfidf+std-cv-10-LR
from sklearn.svm import LinearSVR

lsa_tfidf_std_syn_w2v_lsvr = LinearSVR()

std_scores = cross_val_score(lsa_tfidf_std_syn_w2v_lsvr, X_lsa_tfidf_std_syn_w2v, df.hfdfd, cv=10, scoring='mean_squared_error')

std_mse = abs(std_scores)
print 'LSA+TFIDF+STD+SYN+W2V-cv10-LSVR, mse:', std_mse
print 'LSA+TFIDF+STD+SYN+W2V-cv10-LSVR, mse mean:', np.mean(std_mse)
print 'LSA+TFIDF+STD+SYN+W2V-cv10-LSVR, rmse:', np.sqrt(std_mse)
print 'LSA+TFIDF+STD+SYN+W2V-cv10-LSVR, rmse mean:', np.mean(np.sqrt(std_mse))

In [ ]:
#lsa+tfidf+std-cv-10-LR
from sklearn.svm import SVR

lsa_tfidf_std_syn_w2v_svr = SVR()

std_scores = cross_val_score(lsa_tfidf_std_syn_w2v_svr, X_lsa_tfidf_std_syn_w2v, df.hfdfd, cv=10, scoring='mean_squared_error')

std_mse = abs(std_scores)
print 'LSA+TFIDF+STD+SYN+W2V-cv10-SVR, mse:', std_mse
print 'LSA+TFIDF+STD+SYN+W2V-cv10-SVR, mse mean:', np.mean(std_mse)
print 'LSA+TFIDF+STD+SYN+W2V-cv10-SVR, rmse:', np.sqrt(std_mse)
print 'LSA+TFIDF+STD+SYN+W2V-cv10-SVR, rmse mean:', np.mean(np.sqrt(std_mse))